<a href="https://colab.research.google.com/github/nurfnick/Data_Viz/blob/main/07_SQL_Wrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Wrap Up

To be honest, I am not sure what else to show you.  Not that there isn't more to cover but that I have exhausted my knowledge.

What I think we should do today is to challenge one another.  We are familiar with some of the datasets, can we challenge one another with questions about the data.



In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


I'll start with an example.

How many bottles in each category cost more than two standard deviations above the mean?

First I'll gather the stats of mean and standard deviation.

In [ ]:
%%bigquery --project pic-math

SELECT category_name, AVG(state_bottle_retail) as average, STDDEV(state_bottle_retail) as standarddeviation 
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY category_name

,category_name,average,standarddeviation
0,Imported Distilled Spirit Specialty,14.243394,13.516626
1,Triple Sec,4.073485,0.836835
2,BUTTERSCOTCH SCHNAPPS,9.246485,1.872501
3,IMPORTED DRY GINS,22.195976,6.685395
4,IRISH WHISKIES,25.025809,12.813551
...,...,...,...
131,MISC. IMPORTED CORDIALS & LIQUEURS,21.172184,9.468689
132,SCHNAPPS - IMPORTED,17.758824,24.083047
133,American Whiskies,28.500000,NaN
134,Imported Whiskies,91.980000,22.650188


I am going to use the above table to join with the full table and make the comparison.

In [ ]:
%%bigquery --project pic-math

WITH statsTable as(
SELECT category_name, AVG(state_bottle_retail) as average, STDDEV(state_bottle_retail) as standarddeviation 
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY category_name
)

SELECT t.category_name, COUNT(*) as bottles_over_two_sd
FROM `bigquery-public-data.iowa_liquor_sales.sales` t JOIN statsTable 
      ON t.category_name = statsTable.category_name
WHERE t.state_bottle_retail > statsTable.average + 2*statsTable.standarddeviation
GROUP BY category_name

,category_name,bottles_over_two_sd
0,Neutral Grain Spirits Flavored,1182
1,IMPORTED DRY GINS,10258
2,Triple Sec,5063
3,AMERICAN AMARETTO,5398
4,COFFEE LIQUEURS,2771
...,...,...
109,WHITE CREME DE MENTHE,2
110,WHITE CREME DE CACAO,1
111,SCHNAPPS - IMPORTED,1
112,DARK CREME DE CACAO,1


I have a result but I see several issues here.  One I am counting the same bottles over and over again.  I think I need to group by the *item_description* before I take an average over the category.

In [ ]:
%%bigquery --project pic-math

SELECT category_name, item_description, AVG(state_bottle_retail) as average, STDDEV(state_bottle_retail) as standarddeviation 
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY category_name, item_description

,category_name,item_description,average,standarddeviation
0,Triple Sec,Tortilla Triple Sec,4.520000,0.000000
1,BUTTERSCOTCH SCHNAPPS,Dekuyper Buttershots,11.354986,0.239511
2,IMPORTED DRY GINS,Tanqueray Gin,21.887818,7.089758
3,Coffee Liqueurs,Kahlua Coffee Liqueur,20.277870,6.306792
4,Coffee Liqueurs,Kahlua Coffee,20.795766,6.624718
...,...,...,...,...
13825,Imported Distilled Spirit Specialty,Plantation Trinidad Rum 2005,49.500000,NaN
13826,Imported Distilled Spirit Specialty,Tequila Anejo 750ml Fenice,69.240000,NaN
13827,Imported Distilled Spirit Specialty,Vikre Øvrevann Aquavit,29.010000,NaN
13828,Imported Distilled Spirit Specialty,Amrita Indian Whiskey 375ml Pilar,56.450000,0.000000


That did not work as I'd have hoped (and took a long time to run!)  Instead I am going to gather the `MAX` price of each bottle by *item_description* and use that.  It won't give perfect statistics but it will be better than what I have done.

In [ ]:
%%bigquery --project pic-math

SELECT category_name, item_description, MAX(state_bottle_retail) as retail_max
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY category_name, item_description

,category_name,item_description,retail_max
0,Coffee Liqueurs,Kahlua Coffee,39.72
1,Triple Sec,Juarez Triple Sec,4.01
2,IMPORTED DRY GINS,Bombay Sapphire Gin,39.75
3,Triple Sec,Montezuma Triple Sec,3.20
4,Triple Sec,Dekuyper Triple Sec,3.50
...,...,...,...
13825,Imported Distilled Spirit Specialty,Amor Mio Blanco,45.00
13826,Imported Distilled Spirit Specialty,El Jolgorio Madrecuixe,97.50
13827,Imported Distilled Spirit Specialty,Bobbys Schiedam Jenever,217.50
13828,Imported Distilled Spirit Specialty,Kujira 12 Year Single Grain Ryukyu Whisky,158.00


In [ ]:
%%bigquery --project pic-math

WITH statsTable as(
SELECT category_name, AVG(retail_max) as average, STDDEV(retail_max) as standarddeviation 
FROM (SELECT category_name, item_description, MAX(state_bottle_retail) as retail_max
      FROM `bigquery-public-data.iowa_liquor_sales.sales`
      GROUP BY category_name, item_description) 
GROUP BY category_name
)

SELECT t.category_name, COUNT(*) as bottles_over_two_sd
FROM `bigquery-public-data.iowa_liquor_sales.sales` t JOIN statsTable 
      ON t.category_name = statsTable.category_name
WHERE t.state_bottle_retail > statsTable.average + 2*statsTable.standarddeviation
GROUP BY category_name
ORDER BY bottles_over_two_sd

,category_name,bottles_over_two_sd
0,JAPANESE WHISKY,1
1,Triple Sec,1
2,BARBADOS RUM,1
3,Mezcal,1
4,FLAVORED RUM,1
...,...,...
94,PEACH SCHNAPPS,4659
95,Flavored Rum,4939
96,American Flavored Vodka,5746
97,Whiskey Liqueur,7330


Can you improve the way I have done this?  Does it make sense that there are that many bottles of Whiskey?

I am just going to see if these are correct.  Let's see if we can get the table to include total number of bottles.

In [ ]:
%%bigquery --project pic-math

WITH statsTable as(
SELECT category_name, AVG(retail_max) as average, STDDEV(retail_max) as standarddeviation 
FROM (SELECT category_name, item_description, MAX(state_bottle_retail) as retail_max
      FROM `bigquery-public-data.iowa_liquor_sales.sales`
      GROUP BY category_name, item_description) 
GROUP BY category_name
)

SELECT t.category_name, 
        COUNTIF(t.state_bottle_retail > statsTable.average + 2*statsTable.standarddeviation) as bottles_over_two_sd, 
        COUNT(*) as total_bottles, 
        AVG(statsTable.average)as average_price, 
        AVG(statsTable.standarddeviation) as standard_deviation_price
FROM `bigquery-public-data.iowa_liquor_sales.sales` t JOIN statsTable 
      ON t.category_name = statsTable.category_name
GROUP BY category_name
ORDER BY bottles_over_two_sd

,category_name,bottles_over_two_sd,total_bottles,average_price,standard_deviation_price
0,American Sloe Gins,0,4252,11.630000,7.000000
1,ROOT BEER SCHNAPPS,0,13615,11.148889,2.979188
2,APPLE SCHNAPPS,0,33099,9.493333,4.022831
3,PEACH BRANDIES,0,15070,8.916667,0.962432
4,BLACKBERRY BRANDIES,0,59714,9.895333,4.233651
...,...,...,...,...,...
130,PEACH SCHNAPPS,4659,63400,9.462500,3.478067
131,Flavored Rum,4939,328127,14.767358,5.696967
132,American Flavored Vodka,5746,637286,12.087783,5.055311
133,Whiskey Liqueur,7330,651397,20.977664,22.925363


Yeah maybe that is a reasonable number of whiskeys...

If the price of the bottles was normally distributed, we'd only expect 2.5% but I doubt the price is normal.  Using the mean and standard deviation we see that it is clearly a funky distribution.  

## Your Turn

Challenge your mates with a difficult question.  Try to stump them and don't forget to try it yourself!  For the quiz, pair (or thruple, three or less please!) up with someone in the class and try to come up with a challenge for you both to tackle.  I'd prefer you consider the datasets we have already looked at; liquors, bikeshare (stations or trips) and census.  Try to pose the challenge clearly and see if you can solve it.  Only new questions please!

### Examples



Please only use these if you are really stumped on what to ask!

1. What zip code consumes the most liquor per capita in Iowa?
2. Which zip code uses the bikeshare the most per capita in Austin?
3. Does the percentage of children in an area effect the amount of liquor consumed?  (You need the per capita consumption for this to be accurate!)
4. What type of bikeshare station had the most trips going over a day?